In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ikan import KANLinear, KAN #你可以这样召唤出你需要的KAN

In [3]:
import torch
from torchinfo import summary


# 定义一个 KAN 模型实例
kan_model = KAN(
    layers_hidden=[3, 64, 128, 1],  # 输入维度为 3，隐藏层维度为 64 和 128，输出维度为 1
    grid_size=5,
    spline_order=3,
    scale_noise=0.1,
    scale_base=1.0,
    scale_spline=1.0,
    base_activation=torch.nn.SiLU,
    grid_eps=0.02,
    grid_range=[-1, 1],
)

# 打印模型摘要
summary(kan_model, input_size=(3,))


Layer (type:depth-idx)                   Output Shape              Param #
KAN                                      [1]                       --
├─ModuleList: 1-1                        --                        --
│    └─KANLinear: 2-1                    [64]                      1,920
│    │    └─SiLU: 3-1                    [1, 3]                    --
│    └─KANLinear: 2-2                    [128]                     81,920
│    │    └─SiLU: 3-2                    [1, 64]                   --
│    └─KANLinear: 2-3                    [1]                       1,280
│    │    └─SiLU: 3-3                    [1, 128]                  --
Total params: 85,120
Trainable params: 85,120
Non-trainable params: 0
Total mult-adds (M): 0
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

从输出结果来看，模型结构和参数设置显示了一些细节，显示了每一层的输出形状和参数数量。以下是各层和设置之间的具体联系和解释：

### 模型设置和摘要的联系

#### 设置
```python
kan_model = KAN(
    layers_hidden=[3, 64, 128, 1],  # 输入维度为 3，隐藏层维度为 64 和 128，输出维度为 1
    grid_size=5,
    spline_order=3,
    scale_noise=0.1,
    scale_base=1.0,
    scale_spline=1.0,
    base_activation=torch.nn.SiLU,
    grid_eps=0.02,
    grid_range=[-1, 1],
)
```

#### 模型摘要
```
==========================================================================================
Layer (type:depth-idx)                   Output Shape              Param #
==========================================================================================
KAN                                      [1]                       --
├─ModuleList: 1-1                        --                        --
│    └─KANLinear: 2-1                    [64]                      1,920
│    │    └─SiLU: 3-1                    [1, 3]                    --
│    └─KANLinear: 2-2                    [128]                     81,920
│    │    └─SiLU: 3-2                    [1, 64]                   --
│    └─KANLinear: 2-3                    [1]                       1,280
│    │    └─SiLU: 3-3                    [1, 128]                  --
==========================================================================================
Total params: 85,120
Trainable params: 85,120
Non-trainable params: 0
Total mult-adds (M): 0
==========================================================================================
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
==========================================================================================
```

### 解释

#### KAN 模型实例

- `layers_hidden=[3, 64, 128, 1]`：
  - 模型的输入特征数为 3。
  - 第一层的输出特征数为 64。
  - 第二层的输出特征数为 128。
  - 最后一层的输出特征数为 1。

#### 各层的参数数量

1. **第一层 KANLinear**
   - 输入特征数为 3，输出特征数为 64。
   - 参数数量计算：
     - 基础权重：`3 * 64 = 192`。
     - 样条权重：`(3 * (5 + 3)) * 64 = 3 * 8 * 64 = 1536`。
     - 总参数数量：`192 + 1536 = 1728`（可能略有差异，因为模型中可能还有其他小的参数或偏置项）。

2. **第二层 KANLinear**
   - 输入特征数为 64，输出特征数为 128。
   - 参数数量计算：
     - 基础权重：`64 * 128 = 8192`。
     - 样条权重：`(64 * (5 + 3)) * 128 = 64 * 8 * 128 = 65536`。
     - 总参数数量：`8192 + 65536 = 73728`。

3. **第三层 KANLinear**
   - 输入特征数为 128，输出特征数为 1。
   - 参数数量计算：
     - 基础权重：`128 * 1 = 128`。
     - 样条权重：`(128 * (5 + 3)) * 1 = 128 * 8 * 1 = 1024`。
     - 总参数数量：`128 + 1024 = 1152`。

这些参数数量的计算结果与模型摘要中的参数数量一致。

#### 总参数数量
- 模型的总参数数量为 85,120。

#### 各层的输出形状
- 输入特征数为 3。
- 第一层的输出形状为 `[64]`。
- 第二层的输出形状为 `[128]`。
- 第三层的输出形状为 `[1]`。


In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 计算并打印模型参数数量
total_params = count_parameters(kan_model)
print(f"Total trainable parameters: {total_params}")

# 打印每一层的参数数量
for name, param in kan_model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.numel()} parameters")

# 打印每一层的形状
for name, module in kan_model.named_modules():
    if isinstance(module, KANLinear):
        print(f"{name}: {module}")

# 示例输入
x = torch.randn(1, 3)
# 前向传播
output = kan_model(x)
print(f"Output: {output}")

Total trainable parameters: 85120
layers.0.base_weight: 192 parameters
layers.0.spline_weight: 1536 parameters
layers.0.spline_scaler: 192 parameters
layers.1.base_weight: 8192 parameters
layers.1.spline_weight: 65536 parameters
layers.1.spline_scaler: 8192 parameters
layers.2.base_weight: 128 parameters
layers.2.spline_weight: 1024 parameters
layers.2.spline_scaler: 128 parameters
layers.0: KANLinear(
  (base_activation): SiLU()
)
layers.1: KANLinear(
  (base_activation): SiLU()
)
layers.2: KANLinear(
  (base_activation): SiLU()
)
Output: tensor([[-0.0071]], grad_fn=<ViewBackward0>)


# 测试

In [6]:
import torch
dummy_input = torch.randn(1, 3)
dummy_output = kan_model(dummy_input)
print(dummy_output)
print(dummy_output.shape)


tensor([[0.0008]], grad_fn=<ViewBackward0>)
torch.Size([1, 1])
